## SVARpy: Fast SVAR-GMM

The core function of the SVARpy package is `SVAR.SVARest(u, estimator)`, which is used to estimate the simultaneous interactions in a non-Gaussian SVAR model. The model is represented by the equation $u_t = B_0 \epsilon_t$, where $u_t$ denotes the reduced form shocks and $\epsilon_t$ represents the structural shocks. The function supports various moment-based estimators for the estimation process.

This file provides an overview of the fast SVAR-GMM method implemented by Keweloh (2021) in the SVARpy package. The fast SVAR-GMM estimation can be performed using the following syntax:

```
SVAR.SVARest(u, estimator = 'GMM_WF')
```
 

In [1]:
# Install required packages

# If required pip install SVARpy package
if True:
    !pip install SVARpy

import numpy as np
import matplotlib.pyplot as plt
import SVAR as SVAR
np.random.seed(0)

## Simulate the SVAR

Simulate reduced form shocks $u_t$ from the SVAR

$$
u_t = B_0 ɛ_t
$$

 with $n$ variables, $T$ observations, and structural shocks $ɛ_t$ generated from a uniform distribution.

In [2]:
n = 2  # Number of variables
T = 250  # Number of observations
# Specitfy B0
B0 = np.eye(n)
# Draw structural shocks
eps = np.empty([T, n])
for i in range(n):
    eps[:, i] = np.random.uniform(low=-np.sqrt(3), high=np.sqrt(3), size=T).T
# Generate reduced form shocks u = B eps
u = np.matmul(B0, np.transpose(eps))
u = np.transpose(u)

## Estimator: Fast SVAR-GMM

The fast SVAR-GMM estimator in Keweloh (2021) aims to find a $B$ matrix, such that the innovations 

$$
e(B)_t := B^{-1} u_t
$$

maximize a non-Gaussianity measures based on the shock skewness and excess kurtosis subject to the constrained that the shocks are uncorrelated with unit variance.
 

The fast SVAR-GMM estimator can be defined as follows:

$$
\hat{B}_T    := \argmax \limits_{B \in   \mathbb{B} }
\sum_{i=1}^{n} \left(\frac{1}{T}\sum_{t=1}^{T} e(B)_{i,t}^3\right)^2 + 
\sum_{i=1}^{n} \left(\frac{1}{T}\sum_{t=1}^{T} e(B)_{i,t}^4-3\right)^2  
\\
\text{s.t. } \frac{1}{T} \sum_{t=1}^{T} e(B)_t e(B)_t' = I.
$$

Maximizing the non-Gaussianity, as measured by skewness and excess kurtosis, using the fast SVAR-GMM estimator is equivalent to minimizing the dependency measured by coskewness and cokurtosis conditions using the SVAR-GMM estimator with a specific weighting matrix , see Keweloh (2021).
However, the number of skewness and kurtosis conditions only increases linearly in the number of variables $n$, while the number of coskewness and cokurtosis conditions increases quickly with the number of variables $n$. Consequently, the fast SVAR-GMM estimator remains computationally efficient even for large SVAR models.

### Run estimation:

The following script performs the estimation of the SVAR model using the fast SVAR-GMM estimator (`estimator='GMM_WF'`) with the reduced form shocks $u_t$. Additional options can be specified by passing them through the prepOptions dictionary.
 

In [3]:
prepOptions = dict()
SVAR_out = SVAR.SVARest(u, estimator='GMM_WF', prepOptions=prepOptions)

Estimator: GMM white fast
Estimator Wopt/Avar: -/Uncorrelated
| SVAR            | Moments       | Tests             | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|---------------|-------------------|-------------------------------|
| T=250           | #second: 0    | WaldRec=1.79      | m=2: 1.0 / 1.0                |
| n=2             | #third: 2     | WaldRec-pval=0.18 | m=3: -0.08 / 0.11             |
| #restrictions:0 | #fourth: 2    | J=nan             | m=4: 1.87 / 1.74              |
| #unknowns:4     | ->loss: -2.87 | J-pval=nan        |                               |
 
|                  |      B(:,1)       |      B(:,2)       |
|------------------|-------------------|-------------------|
|      B(1,:)      |       0.98        |       0.06        |
| (avar/wald/pval) | (0.21/1142.9/0.0) | (0.45/1.79/0.18)  |
|                  |                   |                   |
|      B(2,:)      |       0.01        |       1.04        |
| (avar/wald/pval) |  (0.41/0.06/0.8)  | (

### Output:

The output includes the following information:

- Estimator: The estimator used in the `SVAR.SVARest(u, estimator)` function.

- Estimator Wopt/Avar: The options used to calculate the efficient weighting matrix and the asymptotic variance of the estimator.

- T: Sample size.

- n: Number of variables.

- #restrictions: Number of restrictions on the $B_0$ matrix.

- #unknowns: Number of unknown coefficients in the $B_0$ matrix.

- #second: Number of second-order moment conditions (variance and covariance conditions).

- #third: Number of third-order moment conditions (coskewness conditions).

- #fourth: Number of fourth-order moment conditions (cokurtosis conditions).

- loss: GMM loss at $\hat{B}$.

- WaldRec: Wald test statistic for the null hypothesis that the SVAR is recursive.

- WaldRec-pval: P-value corresponding to the Wald test.

- J: Test statistic of the J-Test.

- J-pval: P-value corresponding to the J-Test.

- E[$e_1^m$] with m=2: Variance $E[e_1^2]$ of all estimated shocks.

- E[$e_1^m$] with m=3: Skewness $E[e_1^3]$ of all estimated shocks.

- E[$e_1^m$] with m=4: Kurtosis $E[e_1^4]$ of all estimated shocks.


The output table displays each element of the estimated matrix $\hat{B}$. Restricted values are denoted by "x" and include the following information for each element:

- avar: Estimated asymptotic variance of the corresponding element of the matrix $\hat{B}$.

- wald: Wald test statistic for the null hypothesis that the corresponding element of the matrix $B_0$ is zero.

- pval: P-value corresponding to the Wald test.

### Options:

This section provides an overview on some available options. 

**Fast SVAR-GMM (asymptotic variance)**

The following code shows how to pass different options for the estimator of $S = \underset{T \rightarrow \infty }{lim} E \left[T g_T(B_0) g_T(B_0)' \right] $ used to estimated the  asymptotic variance of the GMM estimator. 

The asymptotic variance  depends on $S = \underset{T \rightarrow \infty }{lim} E \left[T g_T(B_0) g_T(B_0)' \right] $, where $g_T(B)= \frac{1}{T}\sum_{t=1}^{T} f(B,u_t)$ and $f(B,u_t)$ contains the variance, covariance, coskewness, and cokurtosis conditions which correspond to the fast SVAR-GMM estimator.
However, with higher-order moment conditions, estimating $S$ can be challenging in small samples. The problem is studied in detail in Keweloh (2021b). 

With serially independent and mutually uncorrelated i.i.d shocks it holds that $S= E [f(B_0,u_t)f(B_0,u_t)'] $, see Keweloh (2021b). 
Therefore, $S$ can be estimated using the centered version $\frac{1}{T} \sum_{t=1}^{T} (f(B_0,u_t) - \bar{f})(f(B_0,u_t)- \bar{f})'  $  where $bar{f}$ is the mean of $f(B_0,u_t)$ which is implemented using the option  `'Uncorrelated' `.

With serially and mutually independent i.i.d shocks every element of the $S$ matrix can be decomposed into a sum of higher-order moments, see Keweloh (2021b). 
This option is implemented using the option  `'Independent' `.

 

In [7]:
# Use the assumption of serially and mutually  independent shocks to estimate the asymptotic variance (Avarparametric=Independent) , see Keweloh (2021).
prepOptions = dict()
prepOptions['Avarparametric'] = 'Independent'
SVAR_out1 = SVAR.SVARest(u, estimator='GMM_WF', prepOptions=prepOptions)

# Use the assumption of serially independent and mutually uncorrelated shocks to estimate the asymptotic variance (Avarparametric=Uncorrelated) , see Keweloh (2021).
prepOptions = dict()
prepOptions['Avarparametric'] = 'Uncorrelated'
SVAR_out1 = SVAR.SVARest(u, estimator='GMM_WF', prepOptions=prepOptions)

 

Estimator: GMM white fast
Estimator Wopt/Avar: -/Independent
| SVAR            | Moments       | Tests            | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|---------------|------------------|-------------------------------|
| T=250           | #second: 0    | WaldRec=1.63     | m=2: 1.0 / 1.0                |
| n=2             | #third: 2     | WaldRec-pval=0.2 | m=3: -0.08 / 0.11             |
| #restrictions:0 | #fourth: 2    | J=nan            | m=4: 1.87 / 1.74              |
| #unknowns:4     | ->loss: -2.87 | J-pval=nan       |                               |
 
|                  |       B(:,1)       |      B(:,2)       |
|------------------|--------------------|-------------------|
|      B(1,:)      |        0.98        |       0.06        |
| (avar/wald/pval) | (0.21/1142.97/0.0) |  (0.5/1.63/0.2)   |
|                  |                    |                   |
|      B(2,:)      |        0.01        |       1.04        |
| (avar/wald/pval) |  (0.45/0.06/0.81)  | (

**Two-step SVAR-GMM (B startvalues)**

The following code shows how to specify a start value for the B matrix used in the optimization algorithm of the GMM estimator.

In [15]:
# Start at the B matrix obtained from a recursive identification approach (default)
prepOptions = dict()
prepOptions['bstartopt'] = 'Rec'
SVAR_out = SVAR.SVARest(u, estimator='GMM_WF', prepOptions=prepOptions)
 

# Pass a specific B matrix as starting value
prepOptions = dict()
prepOptions['bstartopt'] = 'specific'
prepOptions['bstart'] = np.zeros(int(n*(n-1)/2))  
SVAR_out = SVAR.SVARest(u, estimator='GMM_WF', prepOptions=prepOptions)

Estimator: GMM white fast
Estimator Wopt/Avar: -/Uncorrelated
| SVAR            | Moments       | Tests             | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|---------------|-------------------|-------------------------------|
| T=250           | #second: 0    | WaldRec=1.79      | m=2: 1.0 / 1.0                |
| n=2             | #third: 2     | WaldRec-pval=0.18 | m=3: -0.08 / 0.11             |
| #restrictions:0 | #fourth: 2    | J=nan             | m=4: 1.87 / 1.74              |
| #unknowns:4     | ->loss: -2.87 | J-pval=nan        |                               |
 
|                  |      B(:,1)       |      B(:,2)       |
|------------------|-------------------|-------------------|
|      B(1,:)      |       0.98        |       0.06        |
| (avar/wald/pval) | (0.21/1142.9/0.0) | (0.45/1.79/0.18)  |
|                  |                   |                   |
|      B(2,:)      |       0.01        |       1.04        |
| (avar/wald/pval) |  (0.41/0.06/0.8)  | (

## References

Keweloh, Sascha Alexander. "A generalized method of moments estimator for structural vector autoregressions based on higher moments." Journal of Business & Economic Statistics 39.3 (2021): 772-782.

Keweloh, Sascha Alexander. "A feasible approach to incorporate information in higher moments in structural vector autoregressions." (2021b).